<a href='https://www.darshan.ac.in/'>
  <img src='https://www.darshan.ac.in/Content/media/DU_Logo.svg' width="250" height="300"/>
</a>

<center>
  <h1><b>Data Mining</b></h1>
  <h1><b>Implement Apriori Algorithm on Online Retail Dataset</b></h1>
</center>

---

### **Name:** Harmik Rathod  
### **Enrollment No:** 24010101680  
### **Subject Code:** 2301CS503  
### **Course:** B.Tech in Computer Science & Engineering  
### **Semester:** 5  
### **Division:** B  
 
### **Instructor:** Prof. Jayesh Vagadiya 

---


# Apriori Algorithm Implementation Assignment

### Objective:
You will implement the **Apriori algorithm** from scratch (i.e., without using any libraries like `mlxtend`) to find frequent itemsets and generate association rules.

### Dataset:
Use the [Online Retail Dataset](https://www.kaggle.com/datasets/vijayuv/onlineretail) from Kaggle. You can filter it for a specific country (e.g., `United Kingdom`) and time range to reduce size if needed.

---

## Step 1: Data Preprocessing

- Load the dataset
- Remove rows with missing values
- Filter out rows where `Quantity <= 0`
- Convert Data into Basket Format

👉 **Implement code below**

In [3]:
import pandas as pd
# Load the dataset
df=pd.read_csv('OnlineRetail.csv', encoding='ISO-8859-1')

# Preprocess as per the instructions above | We have already done in TASK 2
df = df.dropna()

df = df[df['Quantity'] > 0]

# for Basket
basket = df.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

C:\Users\HP\AppData\Local\Temp\ipykernel_8556\2371037997.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  basket = basket.applymap(lambda x: 1 if x > 0 else 0)


## Step 2: Implement Apriori Algorithm
Step-by-Step Procedure:
1. Generate Frequent 1-Itemsets
Count the frequency (support) of each individual item in the dataset.
Keep only those with support ≥ min_support.
→ Result is L1 (frequent 1-itemsets)
2. Iterative Candidate Generation (k = 2 to n)
While L(k-1) is not empty:
a. Candidate Generation

Generate candidate itemsets Ck of size k from L(k-1) using the Apriori property:
Any (k-itemset) is only frequent if all of its (k−1)-subsets are frequent.
b. Prune Candidates
Eliminate candidates that have any (k−1)-subset not in L(k-1).
c. Count Support
For each transaction, count how many times each candidate in Ck appears.
d. Generate Frequent Itemsets
Form Lk by keeping candidates from Ck that meet the min_support.
Repeat until Lk becomes empty.
Implement the following functions:
1. `get_frequent_itemsets(transactions, min_support)` - Returns frequent itemsets and their support
2. `generate_candidates(prev_frequent_itemsets, k)` - Generates candidate itemsets of length `k`
3. `calculate_support(transactions, candidates)` - Calculates the support count for each candidate

**Write reusable functions** for each part of the algorithm.

In [4]:
from itertools import combinations

def get_frequent_itemsets(transactions, min_support):
    n = len(transactions)
    
    # Step 1: frequent 1-itemsets
    item_counts = {}
    for t in transactions:
        for item in t:
            key = frozenset([item])
            item_counts[key] = item_counts.get(key, 0) + 1
    
    L1 = {itemset: count / n for itemset, count in item_counts.items() if (count / n) >= min_support}
    frequent_itemsets = dict(L1)

    prev_L = list(L1.keys())
    k = 2
    
    # Step 2: generate larger itemsets
    while prev_L:
        candidates = generate_candidates(prev_L, k)
        candidate_support = calculate_support(transactions, candidates)
        
        current_L = {itemset: support for itemset, support in candidate_support.items() if support >= min_support}
        
        frequent_itemsets.update(current_L)
        prev_L = list(current_L.keys())
        k += 1
    
    return frequent_itemsets


def generate_candidates(prev_frequent_itemsets, k):
    prev_list = [frozenset(x) for x in prev_frequent_itemsets]
    prev_set = set(prev_list)
    candidates = set()

    for i in range(len(prev_list)):
        for j in range(i + 1, len(prev_list)):
            union = prev_list[i] | prev_list[j]
            if len(union) == k:
                # prune: all (k-1)-subsets must be frequent
                if all(frozenset(sub) in prev_set for sub in combinations(union, k - 1)):
                    candidates.add(frozenset(union))
    
    return list(candidates)


def calculate_support(transactions, candidates):
    candidates = [frozenset(c) for c in candidates]
    support_count = {c: 0 for c in candidates}
    n = len(transactions)
    
    for t in transactions:
        for c in candidates:
            if c.issubset(t):
                support_count[c] += 1
    
    return {c: support_count[c] / n for c in candidates}


## Step 3: Generate Association Rules

- Use frequent itemsets to generate association rules
- For each rule `A => B`, calculate:
  - **Support**
  - **Confidence**
- Only return rules that meet a minimum confidence threshold (e.g., 0.5)

👉 **Implement rule generation function below**

In [5]:
# Function to generate rules from frequent itemsets

def generate_rules(frequent_itemsets, min_confidence):
    rules = []

    for itemset, itemset_support in frequent_itemsets.items():
        # only consider itemsets of size >= 2
        if len(itemset) < 2:
            continue

        items = list(itemset)
        # generate all non-empty proper antecedents
        for r in range(1, len(items)):
            for antecedent_tuple in combinations(items, r):
                antecedent = frozenset(antecedent_tuple)
                consequent = itemset - antecedent

                # antecedent support must be available (Apriori guarantees it usually is)
                antecedent_support = frequent_itemsets.get(antecedent)
                if not antecedent_support:
                    continue

                confidence = itemset_support / antecedent_support
                if confidence >= min_confidence:
                    rules.append({
                        'antecedent': set(antecedent),
                        'consequent': set(consequent),
                        'support': itemset_support,
                        'confidence': confidence
                    })

    return rules

## Step 4: Output and Visualize

- Print top 10 frequent itemsets
- Print top 10 association rules (by confidence or lift)

👉 **Output results below**

In [ ]:
# Output the final results
# Optional: Add visualizations

frequent_itemsets = get_frequent_itemsets(basket, min_support=0.01)
rules = generate_rules(frequent_itemsets, min_confidence=0.5)

In [ ]:
# Output Top 10 frequent itemsets
print("Top 10 Frequent Itemsets:")
for i, (items, count) in enumerate(sorted(frequent_itemsets.items(), key=lambda x: x[1], reverse=True)[:10]):
    print(f"{set(items)} - Count: {count}")

In [ ]:

# Output Top 10 rules
print("\nTop 10 Association Rules:")
for i, (antecedent, consequent, confidence, lift) in enumerate(sorted(rules, key=lambda x: x[2], reverse=True)[:10]):
    print(f"{set(antecedent)} -> {set(consequent)} | Confidence: {confidence:.2f} | Lift: {lift:.2f}")